<a href="https://colab.research.google.com/github/YuXuan20040221/GDpj/blob/develop/RoadDamage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 道路安全
* 打 "*" 號的地方是剛開始時需要執行的
* 換資料集
* 調整參數請到"訓練"(旁邊的目錄有寫)

# *掛載 Google Drive
讓 Colab 讀取放在雲端硬碟的資料集
(這邊不用動)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 前置步驟
(安裝YOLOv8)
```
# drive目錄位置
MyDrive/datasets/train.yaml
MyDrive/datasets/images/...
MyDrive/datasets/labels/...

```

路徑:
```
/data_path = "/content/drive/MyDrive/datasets/train.yaml"

```
安裝YOLOv8:

In [ ]:
!pip install ultralytics

# yaml

## 資料集設定

In [ ]:
# 定義train.yaml ->
yaml_content = """
path: /content/drive/MyDrive/datasets  # 整體資料集根目錄
train: Processed_4/train/images # 訓練用資料夾位置
val: Processed_4/valid/images # 驗證用資料夾位置

nc: 1 # 類別個數
names: ["pothole"] # 類別名稱
"""

# 寫入雲端 ->
with open("/content/drive/MyDrive/datasets/train.yaml", "w") as f:
    f.write(yaml_content)

## 模型架構設定:
(這我自己在玩的不用鳥他)

In [ ]:
# 參考自: https://github.com/ultralytics/ultralytics/blob/main/ultralytics/cfg/models/v8/yolov8.yaml
# 僅調整head部分(backbone不動)

# yaml檔 ->
yaml_content = """
# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Ultralytics YOLOv8 object detection model with P3/8 - P5/32 outputs
# Model docs: https://docs.ultralytics.com/models/yolov8
# Task docs: https://docs.ultralytics.com/tasks/detect

# Parameters
nc: 80 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 129 layers, 3157200 parameters, 3157184 gradients, 8.9 GFLOPS
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 129 layers, 11166560 parameters, 11166544 gradients, 28.8 GFLOPS
  m: [0.67, 0.75, 768] # YOLOv8m summary: 169 layers, 25902640 parameters, 25902624 gradients, 79.3 GFLOPS
  l: [1.00, 1.00, 512] # YOLOv8l summary: 209 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPS
  x: [1.00, 1.25, 512] # YOLOv8x summary: 209 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPS

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]] # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]] # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)
"""

# 寫入雲端硬碟檔案 ->
with open("/content/drive/MyDrive/datasets/custom_yolov8n.yaml", "w") as f:
    f.write(yaml_content)

# 訓練
參數調整參考:
[YOLO調整指南](https://docs.ultralytics.com/zh/guides/hyperparameter-tuning/)

In [ ]:
import os
import torch
from ultralytics import YOLO

# 建立模型
model = YOLO("yolov8n.pt") # 模型名稱: 可用YOLO原始模型、自定義模型、之前練過的

# 訓練模型
model.train(
    # 訓練用參數
    data="/content/drive/MyDrive/datasets/train.yaml",
    epochs=150, # 圈數
    batch=16, # 單次處理影像數量
    imgsz=640, # 圖片大小
    project="/content/drive/MyDrive/model_n", # 存在哪資料夾
    name="yolov8n_Processed_4_1", # 叫啥名
    lr0=0.001, # 初始學習率(越小收斂速度越慢但越穩)
    lrf=0.01, # 最終學習率(是lr0的幾%，控制下降幅度)
    weight_decay=0.0005, # L2，防overfitting(但調太大模型難學)
    optimizer="AdamW", # 優化器(AdamW好像大家都會用)
    patience=50,
    warmup_epochs=5, # 穩定前幾epoch
    dropout=0.2, # 關掉多少神經元
    device='cuda',

    # 圖片處理
    hsv_h=0.015,     # 色相擾動
    hsv_s=0.7,       # 飽和度擾動
    hsv_v=0.4,       # 亮度擾動
    degrees=10.0,    # 旋轉
    translate=0.1,   # 平移
    scale=0.3,       # 縮放
    shear=5.0,       # 剪切
)


Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/datasets/train.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_Processed_4_13, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=50, perspective=0.0, pl

train: Scanning /content/drive/MyDrive/datasets/Processed_4/train/labels... 60 images, 1 backgrounds, 0 corrupt: 100%|██████████| 61/61 [00:00<00:00, 159.41it/s]


train: New cache created: /content/drive/MyDrive/datasets/Processed_4/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 1.1±1.1 ms, read: 109.2±65.4 MB/s, size: 544.9 KB)


val: Scanning /content/drive/MyDrive/datasets/Processed_4/valid/labels... 10 images, 0 backgrounds, 0 corrupt: 100%|██████████| 10/10 [00:00<00:00, 85.19it/s]

val: New cache created: /content/drive/MyDrive/datasets/Processed_4/valid/labels.cache


...

                   all         10         25      0.381       0.12     0.0919     0.0333


EarlyStopping: Training stopped early as no improvement observed in last 50 epochs. Best results observed at epoch 74, best model saved as best.pt.
To update EarlyStopping(patience=50) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

124 epochs completed in 0.054 hours.
Optimizer stripped from /content/drive/MyDrive/model_n/yolov8n_Processed_4_13/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/model_n/yolov8n_Processed_4_13/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/model_n/yolov8n_Processed_4_13/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 13.24it/s]


                   all         10         25      0.381       0.16      0.135     0.0501
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /content/drive/MyDrive/model_n/yolov8n_Processed_4_13


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a6a260f9350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# 測試
(不用管他就是我試試看用的)

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/model_n/yolov8n_Processed_2_1_1/weights/best.pt")

results = model.predict(
    source="/content/drive/MyDrive/datasets/Processed_1/valid/images",# ← 測試圖片資料夾
    conf=0.32,          # ← 降低信心閾值，提高 Recall
    iou=0.45,          # ← 放寬 NMS iou
    save=True,          # ← 會把結果圖片存起來
    save_txt=True,        # ← 存文字框框檔
    augment=True         # ← 開啟推論增強（可略）
)

model.val(data="/content/drive/MyDrive/datasets/train.yaml", conf=0.1) # 評估報告


image 1/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000048.jpg: 640x640 1 pothole, 800.2ms
image 2/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: 640x640 5 potholes, 566.8ms
image 3/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000061.jpg: 640x640 1 pothole, 583.2ms
image 4/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000062.jpg: 640x640 4 potholes, 552.3ms
image 5/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000064.jpg: 640x640 1 pothole, 499.6ms
image 6/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000136.jpg: 640x640 (no detections), 550.6ms
image 7/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000160.jpg: 640x640 1 pothole, 480.6ms
image 8/57 /content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000205.jpg: 640x640 3 potholes, 533.9ms
image 9/57 /content/drive/MyDrive/data

100%|██████████| 755k/755k [00:00<00:00, 19.0MB/s]


val: Fast image access ✅ (ping: 0.6±0.2 ms, read: 0.3±0.1 MB/s, size: 62.8 KB)


val: Scanning /content/drive/MyDrive/datasets/China/valid/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:17<00:00,  8.98s/it]


                   all         25         34      0.596      0.348      0.422        0.2
Speed: 17.5ms preprocess, 295.5ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7faf6e7b7390>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

## 測overfitting

In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/model_n/yolov8n_Origin_3_2/weights/best.pt")  # 換成你的模型路徑


# 訓練集評估
train_metrics = model.val(data="/content/drive/MyDrive/datasets/train.yaml", split="train", conf=0.001, iou=0.6)

# 驗證集評估
val_metrics = model.val(data="/content/drive/MyDrive/datasets/train.yaml", split="val", conf=0.001, iou=0.6)

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

# 訓練集 metrics
train_precision = train_metrics.results_dict['metrics/precision(B)']
train_recall = train_metrics.results_dict['metrics/recall(B)']
train_f1 = compute_f1(train_precision, train_recall)

# 驗證集 metrics
val_precision = val_metrics.results_dict['metrics/precision(B)']
val_recall = val_metrics.results_dict['metrics/recall(B)']
val_f1 = compute_f1(val_precision, val_recall)

# 差異比較
print(f"Train F1: {train_f1:.3f}")
print(f"Val F1:   {val_f1:.3f}")
print(f"F1 GAP :  {train_f1 - val_f1:.3f}")


Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 48.0±12.9 MB/s, size: 137.2 KB)


val: Scanning /content/drive/MyDrive/datasets/Origin_3/train/labels.cache... 165 images, 8 backgrounds, 0 corrupt: 100%|██████████| 173/173 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:03<00:00,  2.93it/s]


                   all        173        235       0.97      0.979      0.972       0.56
Speed: 1.2ms preprocess, 6.5ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/val11
Ultralytics 8.3.141 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 27.0±14.5 MB/s, size: 58.1 KB)


val: Scanning /content/drive/MyDrive/datasets/Origin_3/valid/labels.cache... 27 images, 8 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  2.25it/s]


                   all         35         34       0.52      0.588      0.466      0.253
Speed: 10.9ms preprocess, 6.5ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/val12
Train F1: 0.975
Val F1:   0.552
F1 GAP :  0.423


# 投票

## 聯集(OR)

In [ ]:
from ultralytics import YOLO
from pathlib import Path
import torch
import numpy as np

# 載入模型
model_a = YOLO("/content/drive/MyDrive/model_n/yolov8n_Processed_2_1_1/weights/best.pt")
model_b = YOLO("/content/drive/MyDrive/model_n/yolov8n_OriData_4_1/weights/best.pt")

# 預測資料夾
img_dir = Path("/content/drive/MyDrive/datasets/Processed_1/valid/images")
output_dir = Path("/content/drive/MyDrive/datasets/voted_preds/labels")
output_dir.mkdir(parents=True, exist_ok=True)

# -------- Utils --------
def yolo_to_xyxy_tensor(boxes, img_w, img_h):
    """YOLO format (cx, cy, w, h) to XYXY with clamping, tensor version"""
    cx = boxes[:, 0] * img_w
    cy = boxes[:, 1] * img_h
    w = boxes[:, 2] * img_w
    h = boxes[:, 3] * img_h

    x1 = (cx - w / 2).clamp(0, img_w)
    y1 = (cy - h / 2).clamp(0, img_h)
    x2 = (cx + w / 2).clamp(0, img_w)
    y2 = (cy + h / 2).clamp(0, img_h)

    return torch.stack([x1, y1, x2, y2], dim=1)

# 輸出融合後的結果（A or B 模式）
for img_path in img_dir.glob("*.jpg"):
    result_a = model_a.predict(str(img_path), conf=0.32, iou=0.4, verbose=False)[0]
    result_b = model_b.predict(str(img_path), conf=0.353, iou=0.4, verbose=False)[0]

    all_boxes = []

    # 把每個模型的預測框轉換為YOLO格式
    for r in [result_a, result_b]:
        if r.boxes is not None:
            for box in r.boxes:
                cls = int(box.cls[0].item())
                conf = float(box.conf[0].item())
                x1, y1, x2, y2 = box.xyxy[0].tolist()
                x_center, y_center, w, h = box.xywhn[0].tolist()
                all_boxes.append([cls, x_center, y_center, w, h, conf])

    # 把 YOLO 格式轉換為 tensor (XYXY 格式 + conf + cls)
    if all_boxes:
        boxes_tensor = torch.tensor(all_boxes)  # [cls, x_center, y_center, w, h, conf]
        conf = boxes_tensor[:, 5]
        cls = boxes_tensor[:, 0]
        xywh = boxes_tensor[:, 1:5]

        # 轉換為 xyxy
        img_w, img_h = Image.open(img_path).size
        xyxy = yolo_to_xyxy_tensor(xywh, img_w, img_h)

        # 做 NMS（使用 torchvision 自帶的函數）
        import torchvision
        keep = torchvision.ops.nms(xyxy, conf, iou_threshold=0.5)

        # 過濾出保留的框
        filtered_boxes = boxes_tensor[keep]
    else:
        filtered_boxes = torch.zeros((0, 6))


    # 寫入 fusion 預測檔
    label_path = output_dir / f"{img_path.stem}.txt"
    with open(label_path, "w") as f:
        # for box in all_boxes:
        for box in filtered_boxes:
            f.write(f"{box[0]} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f} {box[4]:.6f} {box[5]:.6f}\n")

    result = model_a.predict(img_path, conf=0.3, iou=0.4, save=False, verbose=False)[0]
    for box in result.boxes:
        print(f"xywhn{img_path}:", box.xywhn[0].tolist())


xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000286.jpg: [0.22385621070861816, 0.7224076390266418, 0.1443481296300888, 0.37175655364990234]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: [0.4367590546607971, 0.48660576343536377, 0.09849473834037781, 0.10497763752937317]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: [0.9770104885101318, 0.26265737414360046, 0.045785725116729736, 0.11710286140441895]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: [0.9698253870010376, 0.050241950899362564, 0.06022912263870239, 0.0781131461262703]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000057.jpg: [0.4313870072364807, 0.7975969314575195, 0.07193639874458313, 0.0777050256729126]
xywhn/content/drive/MyDrive/datasets/Processed_1/valid/images/China_Drone_000160.jpg: [0.9319285154342651, 0.2945277690887451, 0.06749075651168823, 0.06

## 驗證

In [ ]:
from pathlib import Path
import torch
from ultralytics.utils.metrics import ConfusionMatrix
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# -------- Utils --------
def yolo_to_xyxy_tensor(boxes, img_w, img_h):
    """YOLO format (cx, cy, w, h) to XYXY with clamping, tensor version"""
    cx = boxes[:, 0] * img_w
    cy = boxes[:, 1] * img_h
    w = boxes[:, 2] * img_w
    h = boxes[:, 3] * img_h

    x1 = (cx - w / 2).clamp(0, img_w)
    y1 = (cy - h / 2).clamp(0, img_h)
    x2 = (cx + w / 2).clamp(0, img_w)
    y2 = (cy + h / 2).clamp(0, img_h)

    return torch.stack([x1, y1, x2, y2], dim=1)

# -------- Visualization --------
def visualize_boxes(img_path, gt_boxes, pred_boxes, save_path):
    img = Image.open(img_path).convert("RGB")
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(img)

    # 畫 Ground Truth 框（綠色）
    for box in gt_boxes:
        x1, y1, x2, y2 = box.tolist()
        w, h = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), w, h, linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)

    # 畫 Prediction 框（紅色）
    for box in pred_boxes:
        x1, y1, x2, y2 = box.tolist()
        w, h = x2 - x1, y2 - y1
        rect = patches.Rectangle((x1, y1), w, h, linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)

    ax.axis("off")
    fig.tight_layout()
    fig.savefig(save_path)
    plt.close(fig)


# -------- Init --------
gt_path = Path("/content/drive/MyDrive/datasets/Processed_1/valid/labels")
pred_path = Path("/content/drive/MyDrive/datasets/voted_preds/labels")
img_dir = Path("/content/drive/MyDrive/datasets/Processed_1/valid/images")  # ← 確保這是你圖片目錄

vis_dir = Path("/content/drive/MyDrive/datasets/voted_preds/vis")  # 儲存可視化結果
vis_dir.mkdir(parents=True, exist_ok=True)

cm = ConfusionMatrix(nc=1)  # 只有一個類別

# -------- Evaluation Loop --------
for gt_file in gt_path.glob("*.txt"):
    pred_file = pred_path / gt_file.name
    img_path = img_dir / (gt_file.stem + ".jpg")
    img_w, img_h = Image.open(img_path).size

    # --- Load GT ---
    gt_lines = gt_file.read_text().strip().splitlines()
    gt = torch.tensor([list(map(float, l.split())) for l in gt_lines]) if gt_lines else torch.zeros((0, 5))
    gt_cls = gt[:, 0] if len(gt) else torch.zeros((0,))
    gt_boxes = yolo_to_xyxy_tensor(gt[:, 1:5], img_w, img_h) if len(gt) else torch.zeros((0, 4))

    # --- Load Prediction ---
    if pred_file.exists():
        pred_lines = pred_file.read_text().strip().splitlines()
        pred = torch.tensor([list(map(float, l.split())) for l in pred_lines]) if pred_lines else torch.zeros((0, 6))
        pred_cls = pred[:, 0]
        pred_boxes = yolo_to_xyxy_tensor(pred[:, 1:5], img_w, img_h)
        pred_conf = pred[:, 5]
    else:
        pred_cls = torch.zeros((0,))
        pred_boxes = torch.zeros((0, 4))
        pred_conf = torch.zeros((0,))

    # --- Format Detections ---
    if len(pred_boxes):
        detections = torch.cat([pred_boxes, pred_conf[:, None], pred_cls[:, None]], dim=1)
    else:
        detections = torch.zeros((0, 6))

    # --- Process ---
    if len(gt_boxes) or len(detections):
        cm.process_batch(detections, gt_boxes, gt_cls)
        # --- 可視化目前這張圖片的 GT 和預測框 ---
        save_img_path = vis_dir / f"{gt_file.stem}_vis.jpg"
        visualize_boxes(img_path, gt_boxes, pred_boxes, save_img_path)

    # --- Optional Debug ---
    print(f"\n📄 {gt_file.name}")
    print(f"GT cls:     {gt_cls.shape} {gt_cls}")
    print(f"GT boxes:   {gt_boxes.shape} {gt_boxes}")
    print(f"Detections: {detections.shape}")

# -------- Metrics --------
confmat = cm.matrix
eps = 1e-16
TP = confmat.diagonal()
FP = confmat.sum(0) - TP
FN = confmat.sum(1) - TP

precision = (TP / (TP + FP + eps)).mean()
recall = (TP / (TP + FN + eps)).mean()
f1 = 2 * precision * recall / (precision + recall + eps)

print(f"\n📊 Voting Ensemble Evaluation Results:")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1 Score:  {f1:.3f}")



📄 China_Drone_000048.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[216.,  85., 285., 280.]])
Detections: torch.Size([1, 6])

📄 China_Drone_000435.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[480., 390., 512., 423.]])
Detections: torch.Size([2, 6])

📄 China_Drone_000160.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[ 98.,  32., 136.,  67.]])
Detections: torch.Size([1, 6])

📄 China_Drone_000205.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[134.,  69., 179., 125.]])
Detections: torch.Size([2, 6])

📄 China_Drone_000136.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[271.,  37., 368., 105.]])
Detections: torch.Size([1, 6])

📄 China_Drone_000451.txt
GT cls:     torch.Size([1]) tensor([0.])
GT boxes:   torch.Size([1, 4]) tensor([[ 27., 149.,  64., 180.]])
Detections: torch.Size([1, 6])

📄 China_Drone_0